# Ranking by Reordering Method

In American collegiate football, it is traditional for each team to play one game
against every other team in their division. We can use two-sided permutation
Procrustes to use the results to rank the teams.
The data taken from *A. N. Langville, C. D. Meyer, Ranking by Reordering Methods, Princeton
University Press, 2012, Ch. 8, pp. 97–112.* shows the pair-wise relationship for 5 football teams
in the Coastal Division of the Atlantic Coast Conference. In Table 1, each team
is given zero points for a game they lost (Duke lost to every other team in its
division) and the point differential is used for games the team won. (E.g., Miami
beat Duke by 45 points and UNC by 18 points.) 


|Table 1. Team by team game score differential|
|---------------------------------------|

| Team  | Duke | Miami | UNC | UVA | VT |
|-------|------|-------|-----|-----|----|
| Duke  | 0    | 0     | 0   | 0   | 0  |
| Miami | 45   | 0     | 18  | 8   | 20 |
| UNC   | 3    | 0     | 0   | 2   | 0  |
| UVA   | 31   | 0     | 0   | 0   | 0  |
| VT    | 45   | 0     | 27  | 38  | 0  |

That is, we can structure the results of the games between the teams using the score-differential matrix,

\begin{equation}
    \mathbf{D} =
    \begin{bmatrix}
        0    &   0    &   0   &    0    &    0 \\
       45    &   0    &  18   &    8    &   20 \\
        3    &   0    &   0   &    2    &    0 \\
       31    &   0    &   0   &    0    &    0 \\
       45    &   0    &   27  &   38    &    0 \\
    \end{bmatrix}
\end{equation}

To rank the teams in the division, we use a ranking vector, which is a permutation of the integers from 1 to n that ranks all the teams. For example, if we list
our teams in the order they appear in Table 1, the vector $[1,3,4,5,2 ] ^{\mathbf{T}}$ assigns Duke with rank position 1, VT with rank position 2, etc. To use the permutation Procrustes method, we need to express the ranking vector as a matrix; to do this we define an $n \times n$ rank-differential matrix which is a symmetric reordering of the fundamental rank-differential matrix $\hat{\mathbf{R}} \in \mathbb{R}^{n \times n}$,

\begin{equation}
    \hat{\mathbf{R}}_{n \times n} =
    \begin{bmatrix}
        0 & 1 & 2 & \cdots & n-1 \\
        & 0 & 1 & \cdots & n-2 \\
        &   &\ddots &\ddots & \vdots \\
        &   &   & \ddots & 1 \\
        &   &   &        & 0
    \end{bmatrix}
\end{equation}

The goal is then to find the optimal permutation, $\mathbf{Q}$, that maximizes the similarity between the rank-differential matrix, $\mathbf{R}$, and the score-differential matrix, $\mathbf{D}$. This is a two-sided permutation Procrustes problem with one transformation,
\begin{equation}
    \min_{\mathbf{Q}} {\left\lVert \mathbf{Q}^{\top} \mathbf{D} \mathbf{Q} - \hat{\mathbf{R}}
        \right\rVert}_{F}^2
\end{equation}

In order to compute the ranking vector, we need the fundamental rank-differential matrix $\hat{\mathbf{R}}_{n \times n}$. So we build a function to compute the rank differential matrix based on the given data,

In [ ]:
# only run this cell if you need to install the dependices
!pip install git+https://github.com/theochem/procrustes.git@master

In [1]:
# import required libraries
import numpy as np
from procrustes import permutation_2sided

def rank_differential(D):
    r""" Compute the rank differential based on the shape of input data."""

    N = np.shape(D)[0]
    R_hat = np.zeros((N, N))
    # Compute the upper triangle part of R_hat
    a = []
    for each in range(N):
        # print(each)
        a.extend(range(0, N-each))
     # Get the R_hat
    R_hat[np.triu_indices_from(R_hat, 0)] = a
    
    return R_hat

Now we can solve the problem by two-sided permutation Procrustes,

In [2]:
def ranking(D, perm_mode='normal1'):
    r"""Compute the ranking vector."""

    #_check_input(D)
    R_hat = rank_differential(D)
    res = permutation_2sided(D, R_hat,
                            remove_zero_col=False,
                            remove_zero_row=False,
                            mode=perm_mode)
    # Compute the rank
    _, rank = np.where(res["t"] == 1)
    rank += 1

    return rank

In [3]:
# the initial data
D = np.array([[ 0, 0, 0 ,  0,  0 ],
              [45, 0, 18,  8,  20], 
              [ 3, 0, 0 ,  2,  0 ],
              [31, 0, 0 ,  0,  0 ],
              [45, 0, 27, 38,  0 ]])

In [4]:
# compute the rank
rank = ranking(D)

Maximum iteration reached! Change=4.783354588670838e-07


In [5]:
print(rank)

[5 2 4 3 1]


Why we need to add all the rank values by 1? Because Python's list index starts with 0, but we often index starting from 1 for physical objects.